# PhaseObjC - Stoichiometric Phase Example 2

Required Python code to load the phase library.

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

### Create a Python reference to the `QuartzBerman` stoichiometric phase class, and instantiate an instance of that class.  
[Documentation for the QuartzBerman class](https://gitlab.com/ENKI-portal/PhaseObjC/blob/master/Documentation/QuartzBerman.md) 

In [ ]:
QuartzBerman = ObjCClass('QuartzBerman')
obj = QuartzBerman.alloc().init()

### Obtain properties of the phase inherited from the `PhaseBase` class.  
Same syntax in Objective-C.  `mw` is the molecular weight in grams/mole.

In [ ]:
print (obj.phaseName)
print (obj.phaseFormula)
print (obj.mw)

Number of moles of each element calculated from the phase formula.  
@property (readonly) DoubleVector *formulaAsElementArray;  

Note that the class method, `elementNameFromAtomicNumber`, from the `PhaseBase` base class is used to construct a formula directly from the moles vector.  

In [ ]:
PhaseBase = ObjCClass('PhaseBase')
moles_pointer = obj.formulaAsElementArray
ne = obj.formulaAsElementArray.size
formula = ''
for i in range(0, ne):
    value = obj.formulaAsElementArray.valueAtIndex_(i)
    if value != 0.0:
        name = PhaseBase.elementNameFromAtomicNumber_(i)
        formula = formula + name + '(' + str(value) + ')'
print (formula)

## All phases that conform to the Stoichiometric Phase Protocol ...
### ...implement the following functions:
```
(double)getGibbsFreeEnergyFromT:(double)t andP:(double)p;
(double)getEnthalpyFromT:(double)t andP:(double)p;
(double)getEntropyFromT:(double)t andP:(double)p;
(double)getHeatCapacityFromT:(double)t andP:(double)p;
(double)getDcpDtFromT:(double)t andP:(double)p;
(double)getVolumeFromT:(double)t andP:(double)p;
(double)getDvDtFromT:(double)t andP:(double)p;
(double)getDvDpFromT:(double)t andP:(double)p;
(double)getD2vDt2FromT:(double)t andP:(double)p;
(double)getD2vDtDpFromT:(double)t andP:(double)p;
(double)getD2vDp2FromT:(double)t andP:(double)p;
```
where *t* (temperature) is in K, and *p* (pressure) is in bars.
### In Python, these calls are written:

In [ ]:
print ("{0:>10s}{1:15.2f}{2:<20s}".format("G", obj.getGibbsFreeEnergyFromT_andP_(1000.0, 1000.0), ' J/mol'))
print ("{0:>10s}{1:15.2f}{2:<20s}".format("H", obj.getEnthalpyFromT_andP_(1000.0, 1000.0), ' J/mol'))
print ("{0:>10s}{1:15.2f}{2:<20s}".format("S", obj.getEntropyFromT_andP_(1000.0, 1000.0), ' J/K-mol'))
print ("{0:>10s}{1:15.3f}{2:<20s}".format("Cp", obj.getHeatCapacityFromT_andP_(1000.0, 1000.0), ' J/K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dCp/dT", obj.getDcpDtFromT_andP_(1000.0, 1000.0), ' J/-K^2-mol'))
print ("{0:>10s}{1:15.3f}{2:<20s}".format("V", obj.getVolumeFromT_andP_(1000.0, 1000.0), ' J/bar-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dV/dT", obj.getDvDtFromT_andP_(1000.0, 1000.0), ' J/bar-K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("dv/dP", obj.getDvDpFromT_andP_(1000.0, 1000.0), ' J/bar^2-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dT2", obj.getD2vDt2FromT_andP_(1000.0, 1000.0), ' J/bar-K^2-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dTdP", obj.getD2vDtDpFromT_andP_(1000.0, 1000.0), ' J/bar^2-K-mol'))
print ("{0:>10s}{1:15.6e}{2:<20s}".format("d2V/dP2", obj.getD2vDp2FromT_andP_(1000.0, 1000.0), ' J/bar^3-mol'))

## Illustrate the Parameter Calibration Protocol.

In [ ]:
try:
    if obj.supportsParameterCalibration() == 1:
        print ('This phase supports the Calibration protocol')
    np = obj.getNumberOfFreeParameters()
    print ('... there are', np, 'parameters')
    names = obj.getArrayOfNamesOfFreeParameters()
    import ctypes
    m = (ctypes.c_double*1)()
    ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
    m[0] =  1.0
    t = 1000.0
    p = 1000.0
    for i in range (0, np):
        name = names.objectAtIndex_(i)
        value = obj.getValueForParameterName_(name)
        units = obj.getUnitsForParameterName_(name)
        dmudw = obj.getChemicalPotentialDerivativesForParameter_usingMolesOfComponents_andT_andP_(name, m, t, p)
        print ("Parameter {0:<15s} = {1:15.6e} {2:<15s}, dmu/dParameter = {3:15.6e}".format(name, value, units, 
                                                                                   dmudw.valueAtIndex_(0)))
except AttributeError:
    print ('This phase does not implement the parameter calibration protocol')